In [1]:
!python -m spacy download en_core_web_md 

     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 393.8 kB/s eta 0:01:49
     ---------------------------------------- 0.2/42.8 MB 1.4 MB/s eta 0:00:31
     ---------------------------------------- 0.5/42.8 MB 2.9 MB/s eta 0:00:15
     - -------------------------------------- 1.2/42.8 MB 5.7 MB/s eta 0:00:08
     -- ------------------------------------- 3.0/42.8 MB 11.1 MB/s eta 0:00:04
     --- ------------------------------------ 4.1/42.8 MB 13.2 MB/s eta 0:00:03
     ----- ---------------------------------- 5.5/42.8 MB 15.2 MB/s eta 0:00:03
     ----- ---------------------------------- 6.1/42.8 MB 14.9 MB/s eta 0:00:03
     ------ --------------------------------- 7.3/42.8 MB 16.1 MB/s eta 0:00:03
     -------- ------------------------------- 8.7/42.8 MB 17.4 MB/s eta 0:00:02
     --------- ------------------------------ 9.7/42.8 MB 1

In [1]:
import os
import csv
from pathlib import Path
import spacy
import en_core_web_sm
import pypandoc
from pypandoc.pandoc_download import download_pandoc
#download_pandoc()


In [2]:
class Skill:
    def __init__(self,name,dir, keyword , groups=None,prerequisites= None):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join(dir, filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = keyword  # keyword for searching LLM
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)
        self.prerequisite  = set()
        if prerequisites is not None:
            self.UpdatePrerequisites(prerequisites)
        

    def UpdateGroupSet(self,groups):
        self.group_set.update(groups)
        #print("skill group set updated.")
      
     
    def UpdatePrerequisites(self,prerequisites):
        self.prerequisite.update(prerequisites)
        #print("prerequisite updated.")
        
class Group:
    def __init__(self,name,skills):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("group", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " in tech" # keyword for searching LLM
        self.skill_set = skills

    def UpdateSkillSet(self,skill):
        self.skill_set.update(skill)
        #print("group skill set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword

    
    
        

In [7]:
class TechStack:
    def __init__(self):
        self.skill_dict_list = {}
        self.group_dict_list = {}
        self.exact_match_replace_dict_list = {}
        self.partial_match_replace_dict_list = {}
        self.ignore_set = set()
        self.not_found_set = set()
        self.document_pepare_set = set()
        self.nlp = spacy.load('en_core_web_md')
        self.ImportIgnoreSet()

               
    def AddSkillDictList(self,name,path,keyword,groups=None,prerequisites = None):
        if name not in self.skill_dict_list:
            self.skill_dict_list[name] = Skill(name,path,keyword,groups,prerequisites)
            #print(name,"added in skill_dict_list.")
            if groups is not None:
                for g in groups:
                    if g in self.group_dict_list:
                        self.group_dict_list.get(g).UpdateSkillSet({name})
                        #print(name,"added in",g,".")
                    else:
                        self.group_dict_list[g] = Group(g,{name})
                        #print("new group:",g,"have been created and added",name,".")
        else:
            self.UpdateSkillDictList(name,groups,prerequisites)
                    
    def UpdateSkillDictList(self,name,groups,prerequisites):
        if name in self.skill_dict_list:
            self.skill_dict_list[name].UpdateGroupSet(groups)
            if prerequisites is not None:
                self.skill_dict_list[name].UpdatePrerequisites(prerequisites)
                #print(name,"prerequisites updated.")

    def AddGroupDictList(self,name,skills):
        if skills is not None:
            if name in self.group_dict_list:
                self.UpdateGroupDictList(name,skills)
            else:
                found_set = set()
                for s in skills:
                    if s in self.skill_dict_list:
                        self.skill_dict_list[s].UpdateGroupSet({name})
                        found_set.add(s)  
                        #print(s,"added in",name,"group set.")
                self.group_dict_list[name] = Group(name,found_set)

    def UpdateGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                      found_set.add(s)  
            self.group_dict_list[name].UpdateSkillSet(found_set)
        else:
            self.AddGroupDictList(name,skills)

    def ImportIgnoreSet(self):
        f = open("ignore.txt", "r")
        for c in f:
            c = c.replace("\n", "")
            self.ignore_set.add(c)
        f.close()

    def ExportSkillDictList(self):
        file_path = "skills.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","Groups","Prerequisites"])
            for key, value in self.skill_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                groups =""
            
                for g in value.group_set:
                    groups += "["
                    groups += g
                    groups +="]"
                pre = ""
                for p in value.prerequisite:
                    pre += "["
                    pre += p
                    pre +="]"
                writer.writerow([name,search,path,groups,pre])
            file.close()

    def ExportGroupDictList(self):
        file_path = "groups.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","skills"])
            for key, value in self.group_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                skills =""
                for s in value.skill_set:
                    skills += "["
                    skills += s
                    skills +="]"
                writer.writerow([name,search,path,skills])
            file.close()



    def ExportMatchReplaceDictList(self):
        file_path = "exact match.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Word", "Replace"])
            for key, value in self.exact_match_replace_dict_list.items():
                writer.writerow([key,value])
            file.close()
        file_path = "partial match.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Word", "Replace"])
            for key, value in self.partial_match_replace_dict_list.items():
                writer.writerow([key,value])
            file.close()



    def ExportNotFoundSet(self):
        with open('not found.txt', 'w') as f:
            for i in  self.not_found_set:
                f.write(i)
                f.write("\n")
            f.close()

    def Filter(self, text):
        text = text.lower()

        if text.find('(') != -1:
            text = text.split("(")[0]
            text = text.rsplit()[0]
            
        words = text.split()
        for word in words:   
            if word in self.ignore_set:
                return str("")
            
        if text in self.exact_match_replace_dict_list:
            text = self.exact_match_replace_dict_list.get(text)
            
        words = text.split()
        new_text = ""
        for word in words:
            if word in self.partial_match_replace_dict_list:
                new_text += self.partial_match_replace_dict_list.get(word) 
                new_text +=" "
            else:
                new_text += word 
                new_text +=" "
        return new_text[:-1]

    def Search(self,text):
        if text in self.skill_dict_list:
            self.document_pepare_set.add(text)
            return True
        if text in self.group_dict_list:
            self.document_pepare_set.add(text)
            return True
        # check . rmeove
        for sdl in  self.skill_dict_list:
            check1 = sdl
            check1 = check1.replace(".","")
            check2 = text
            check2 = check2.replace(".","")
            if check1 == check2:
                self.document_pepare_set.add(sdl)
                return True
        found = False      
        words = text.split()
        # check word by word
        for word in words:   
            if word in self.skill_dict_list:
                self.document_pepare_set.add(word)
                found = True
            elif word in self.group_dict_list:
                self.document_pepare_set.add(word)
                found = True
        
        return found 
        

    def GenerateLearningResource(self,your_skills, job_skills, company_name):
        skills = set()
        if your_skills is not None:
            skills =  job_skills -  your_skills 
        else:
            skills = job_skills

        if len(skills) == 0:
            print("you are good.")
            return False
        
        self.document_pepare_set.clear()

        for s in skills:
            s = self.Filter(s)
            if s != "":
                found = self.Search(s)
                if found == False:
                    self.not_found_set.add(s)
                    
        if len(self.document_pepare_set) == 0 :
            print("No any learning resource generated.")
            return False
        print(self.document_pepare_set)
        html_content = ""
        for d in self.document_pepare_set:
            path = ""
            if d in self.skill_dict_list:
                v = self.skill_dict_list.get(d)
                path = v.resource_path
            elif d in self.skill_dict_list:
                v = self.group_dict_list.get(d)
                path = v.resource_path
            else:
                continue

            if os.path.isfile(path) == False:
                print(d,"not found in",path)
            else:
                with open(path, "r", encoding="utf-8") as file:
                    title = d.title()
                    html_content +="<h1><u><b>"
                    html_content += title
                    html_content +="</b></u></h1>"
                    html_content += file.read()
                file.close()

        output = pypandoc.convert_text(html_content, 'docx', format='html', outputfile='learning resource.docx')
        if output == "":
            print("Document output sucessfully.")
            return True
        else:
            print("Document output failed")
            return False
        
   

In [8]:
test = TechStack()

# top programming list
name_list = []
name_list.append("javascript")
name_list.append("typescript")
name_list.append("html")
name_list.append("css")
name_list.append("python")
name_list.append("bash")
name_list.append("java")
name_list.append("c++")
name_list.append("c")
name_list.append("php")
name_list.append("rust")
name_list.append("kotlin")
name_list.append("ruby")
name_list.append("lua")
name_list.append("dart")
name_list.append("assembly")
name_list.append("r")
name_list.append("matlab")
name_list.append("delphi")
name_list.append("scala")
name_list.append("perl")
name_list.append("elixir")
name_list.append("objective c")
name_list.append("haskell")
name_list.append("gdscript")
name_list.append("lisp")
name_list.append("solidity")
name_list.append("clojure")
name_list.append("julia")
name_list.append("erlang")
name_list.append("fortran")
name_list.append("prolog")
name_list.append("zig")
name_list.append("ada")
name_list.append("ocaml")
name_list.append("sas")
name_list.append("crystal")
name_list.append("apl")
name_list.append("flow")
name_list.append("raku")
name_list.append("racket")
name_list.append("xhtml")
name_list.append("d")
name_list.append("scratch")
name_list.append("sql")
for i in range(len(name_list)):
    keyword = name_list[i] + " programming"
    test.AddSkillDictList(name_list[i], "programming", keyword,{"programming"})

name = "socket"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network"} )
name = "rpc"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network"} )
name = "checksum"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "multiprocessing"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency"} )
name = "multithreading"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency"} )
name = "asynchronous"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency"} )
name = "serialization"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "deserialization"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "file io"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
name = "encrypt"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","security"} )
name = "decrypt"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","security"} )
name = "requests"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network","web"} )
name = "lru cache"
test.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )


# microsoft
name = "windows"
test.AddSkillDictList(name, "microsoft", name , {"microsoft","os"} )
name = "windows server"
test.AddSkillDictList(name, "microsoft", name , {"microsoft","os","server"} )
name = "mircosoft sql"
test.AddSkillDictList(name, "microsoft", "microsoft "+ name , {"microsoft","database","data warehouse","relational"})
name = "word"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "excel"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "powerpoint"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "visio"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "sharepoint"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "dynamics"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "exchange"
test.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
name = "mircosoft visual studio"
test.AddSkillDictList(name, "ide", name, {"microsoft","ide"} )
name = "mircosoft visual code"
test.AddSkillDictList(name, "ide", name, {"microsoft","ide"} )
name= "power bi"
test.AddSkillDictList(name, "microsoft", name ,{"visualization","microsoft"} )
name = "c#"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
name = "f#"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
name = "powershell"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
name = "visual basic"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
name = "visual basic .net"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
name = "vba"
test.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
name = "aspnet mvc"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "asp.net"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "asp.net core"
test.AddSkillDictList(name, "web", name, {"microsoft","framework"} )
name = ".net"
test.AddSkillDictList(name, "web",  ".net", {"microsoft","framework" })
name = ".net core"
test.AddSkillDictList(name, "web", ".net core", {"microsoft","framework" })
name = "wpf"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "linq"
test.AddSkillDictList(name, "web", name, {"microsoft","framework" })
name = "directx"
test.AddSkillDictList(name, "game", name, {"microsoft","framework","game" })
name = "microsoft game developer kit"
test.AddSkillDictList(name, "game", name, {"microsoft","framework","game" })
name = "microsoft quantum"
test.AddSkillDictList(name, "quantum", name , {"microsoft","quantum"})
name = "microsoft system center"
test.AddSkillDictList(name, "microsoft", name , {"microsoft","network","network montioring"})
#azure
name = "cosmosdb"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","database","non-relational"})
name = "azure sql database"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","database","relational"})
name = "azure blobs"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure","storage"})
name = "azure file"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure"})
name = "azure elastic san"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure tables"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure queues"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure"})
name = "azure managed disks"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure container storage"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "azure container"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure","container"})
name = "azure data lake"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
name = "service bus"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","message queue"})
name = "hdinsight"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","message queue"})
name = "azure synapse analytics"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","data warehouse","big data"})
name = "azure ai"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","artificial intelligence"})
name = "azure pipelines"
test.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","continuous integration and continuous deployment"})
name = "arm"
test.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure"})
name = "azure virtual machines"
test.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure", "virtual machine"})
name = "azure montior"
test.AddSkillDictList(name, "microsoft", name,{"microsoft","cloud","azure","reliability","performance monitoring"} )
name = "azure migration"
test.AddSkillDictList(name, "microsoft", name,{"microsoft","cloud","azure","reliability","migration"} )


#google
name= "android"
test.AddSkillDictList(name, "google", name,{"google","os", "mobile"})
name= "go"
test.AddSkillDictList(name, "programming", name + " programming",{"google","programming"})
name= "android studio"
test.AddSkillDictList(name, "mobile", name,{"google","android", "mobile"})
name= "android sdk"
test.AddSkillDictList(name, "mobile", name,{"google","android", "mobile"})
name= "google docs"
test.AddSkillDictList(name, "google", name,{"google"})
name= "google sheets"
test.AddSkillDictList(name, "google", name,{"google"})
name= "google slides"
test.AddSkillDictList(name, "google", name,{"google"})
name= "google analytics"
test.AddSkillDictList(name, "google", name,{"google","web"})
#google cloud
name = "cloud sql"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","relational"})
name = "alloydb"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","relational"})
name = "spanner"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","relational"})
name = "bigtable"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","big data"})
name = "firestore"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","document database"})
name = "firebase realtime database"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","document database"})
name = "memorystore"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","in memory database"})
name = "mongodb atlas"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","non-relational"})
name = "bigquery"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","data warehouse","big data"})
name = "dataproc"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud"})
name = "google cloud storage"
test.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","storage"})
name = "vertex ai"
test.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","artificial intelligence"})
name = "cloud tasks"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","message queue"})
name = "pub sub"
test.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","message queue"})
name = "google kubernetes engine"
test.AddSkillDictList(name, "google",  name, {"google","cloud","google cloud","container"})
name = "google cloud monitoring"
test.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","performance monitoring"})
name = "cloud build"
test.AddSkillDictList(name, "google", "google "+ name, {"google","cloud","google cloud","continuous integration and continuous deployment"})
name = "google compute engine"
test.AddSkillDictList(name, "google",  name, {"google","cloud","google cloud","virtual machine"})

#aws
name = "s3"
test.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","storage"})
name = "fsx"
test.AddSkillDictList(name, "amazon", "amazon " +name, {"cloud","amazon web services"})
name = "ebs"
test.AddSkillDictList(name, "amazon", "amazon " +name, {"cloud","amazon web services","storage"})
name = "amazon file cache"
test.AddSkillDictList(name, "amazon", name, {"cloud","amazon web services"})
name = "drs"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services"})
name = "neptune"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","graph database"})
name = "rds"
test.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","database","relational"})
name = "aurora"
test.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","database","relational"})
name = "dynamodb"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","non-relational"})
name = "redshift"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","data warehouse"})
name = "elasticache"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","data store"})
name = "memorydb"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","in memory database"})
name = "documentdb"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","document databaset"})
name = "timestream"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","time series"})
name = "athena"
test.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","big data"})
name = "emr"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","big data"})
name = "sns"
test.AddSkillDictList(name, "amazon", "amazon " +  name, {"cloud","amazon web services","message queue","big data"})
name = "amazon lake formation"
test.AddSkillDictList(name, "amazon", name, {"cloud","amazon web services"})
name = "sagemaker"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","artificial intelligence"})
name = "ecs"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","container"})
name = "fargate"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","container"})
name = "amazon lambda"
test.AddSkillDictList(name, "amazon",  name, {"cloud","amazon web services"})
name = "amazon vpc"
test.AddSkillDictList(name, "amazon",  name, {"cloud","amazon web services"})
name = "cloudwatch"
test.AddSkillDictList(name, "amazon", "amazon " +  name, {"cloud","amazon web services","performance monitoring"})
name = "sqs"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","message queue"})
name = "amazon codepipeline"
test.AddSkillDictList(name, "amazon",  name, {"cloud","amazon web services","continuous integration and continuous deployment"})
name = "ec2"
test.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services", "virtual machine"})

# apple
name = "mac"
test.AddSkillDictList(name, "apple", name + " technology", {"apple","os"} )
name = "ios"
test.AddSkillDictList(name, "apple", name + " technology", {"apple","os","mobile"} )
name = "swift"
test.AddSkillDictList(name, "programming", name + " programming",{"apple","programming"})
name = "ios sdk"
test.AddSkillDictList(name, "apple", name ,{"apple","framework", "mobile","ios"})
name = "xcode"
test.AddSkillDictList(name, "apple", name ,{"apple","framework"})
# oracle
name = "oracle sql"
test.AddSkillDictList(name, "oracle", name, {"oracle","database"})
name = "pl-sql"
test.AddSkillDictList(name, "programming", name + " programming", {"oracle","database","programming"})
name= "adf"
test.AddSkillDictList(name, "oracle", "oracle "+name + " framework",{"oracle"}  )
name = "oracle cloud"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud"}  )
name = "oracle virtual box"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud", "virtual machine"}  )
name = "oracle c"
test.AddSkillDictList(name, "oracle", name, {"oracle"})
name = "ebs"
test.AddSkillDictList(name, "oracle", "oracle "+name ,{"oracle","cloud"}  )
name = "oracle monitoring"
test.AddSkillDictList(name, "oracle", name ,{"oracle","cloud","performance monitoring","reliability"}  )
name = "soa"
test.AddSkillDictList(name, "oracle", "oracle "+name ,{"oracle","cloud"}  )
name = "epm"
test.AddSkillDictList(name, "oracle", "oracle "+ name ,{"oracle","cloud"}  )
name = "oracle linux"
test.AddSkillDictList(name, "oracle", name ,{"oracle", "cloud"} )
name = "oracle net"
test.AddSkillDictList(name, "oracle", name ,{"oracle", "cloud"} )

#ibm
name = "db2"
test.AddSkillDictList(name, "ibm", "ibm " + name, {"database","ibm"})
name = "netezza"
test.AddSkillDictList(name, "ibm", "ibm " + name, {"database","ibm","warehouse"})
name = "ibm server"
test.AddSkillDictList(name, "ibm", name, {"server","ibm"})
name = "ibm mq"
test.AddSkillDictList(name, "ibm", name, {"message queue","ibm"})
name = "ibm aix"
test.AddSkillDictList(name, "ibm", name, {"performance montior","ibm"})
name = "ibm watson"
test.AddSkillDictList(name, "ibm", name + " framework", {"ibm","artificial intelligence"})
name = "qiskit"
test.AddSkillDictList(name, "ibm", name , {"ibm","quantum"})
name = "z-os"
test.AddSkillDictList(name, "ibm", name , {"ibm","os"})
name = "informix"
test.AddSkillDictList(name, "ibm", name , {"ibm","reliability","migration"})
name = "jcl"
test.AddSkillDictList(name, "ibm", name + " programming", {"ibm","programming"})

# vmware
name = "vsphere"
test.AddSkillDictList(name, "vmware", "vmware " + name, {"cloud","vmware"} )
name = "vcenter"
test.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
name = "esxi"
test.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
name = "vcloud"
test.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
name = "sdn"
test.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
name = "vmware certified professional"
test.AddSkillDictList(name, "vmware", name, {"vmware"} )

#sap
name = "hana"
test.AddSkillDictList(name, "sap", name + " database", {"database","sap"})
name = "sap bi"
test.AddSkillDictList(name, "sap", name , {"visualization","sap"})
name = "sap bw"
test.AddSkillDictList(name, "sap", name , {"data warehouse","sap"})
name = "sap basis"
test.AddSkillDictList(name, "sap", name , {"sap"})
name = "sap abap"
test.AddSkillDictList(name, "sap", name , {"sap", "programming"})

# apache
name = "accumulo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "activemq"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "airavata"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "airflow"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "allura"
test.AddSkillDictList(name, "git", "apache " + name , {"apache","git"})
name = "ambari"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ant"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ivy"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "apisix"
test.AddSkillDictList(name, "microservices", "apache " + name , {"apache","microservices"})
name = "aries"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "arrow"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "avro"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "axis"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "rampart"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "beam"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "bloodhound"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","git"})
name = "brooklyn"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "calcite"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "camel"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "carbondata"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "cassandra"
test.AddSkillDictList(name, "database", "apache " + name , {"apache","database","non-relational"})
name = "cayenne"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "celix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cloudstack"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cacoon"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "commons"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "bcel"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "logging"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ognl"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cordova"
test.AddSkillDictList(name, "mobile", "apache " + name , {"apache"})
name = "couchdb"
test.AddSkillDictList(name, "database", "apache " + name , {"apache","database", "non-relational","document database"})
name = "cxf"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "derby"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","rdbms"})
name = "jdo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "cayenne"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ldap"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "fortress"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "scimple"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "drill"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "druid"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "empire-db"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "felix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "flex"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "flink"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "flume"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "fluo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "freemarker"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "geronimmo"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "groovy"
test.AddSkillDictList(name, "programming", "apache " + name , {"apache","programming"})
name = "gump"
test.AddSkillDictList(name, "continuous integration", "apache " + name , {"apache"})
name = "hadoop"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "hbase"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","database","big data"})
name = "helix"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "hive"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","data warehouse"})
name = "http server"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "iceberg"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "ignite"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "impala"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "iotdb"
test.AddSkillDictList(name, "database", "apache " + name , {"apache","database","time series"})
name = "jackrabbit"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "james"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "jmeter"
test.AddSkillDictList(name, "testing", "apache " + name , {"apache","testing"})
name = "kafka"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "kudu"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "kylin"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "log4j"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "log4net"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "lucene core"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "solr"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "mahout"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","machine learning","artificial intelligence"})
name = "maven"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "doxia"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "messos"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "ftp server"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "mina"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "myface"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "mynewt"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "netbeans"
test.AddSkillDictList(name, "ide", "apache " + name , {"apache","ide"})
name = "nifi"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "nutch"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","web crawler"})
name = "nuttx"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","os"})
name = "ofbix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "oozie"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "openjpa"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "opennlp"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","natural language processing","artificial intelligence"})
name = "openoffice"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "orc"
test.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
name = "parquet"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pdfbox"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "mod perl"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "phoenix"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pig"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pinot"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pivot"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "poi"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "xmlbeans"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "pr"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "qpid"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "rocketmq"
test.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
name = "roller"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "royale"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "samza"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "shiro"
test.AddSkillDictList(name, "security", "apache " + name , {"apache","security"})
name = "singa"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","artificial intelligence"})
name = "sling"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "solr"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "spamassassin"
test.AddSkillDictList(name, "security", "apache " + name , {"apache","security"})
name = "spark"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "storm"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "struts"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "subversion"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache","git"})
name = "superset"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "synapse"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "systemds"
test.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","machine learning","artificial intelligence"})
name = "tapestry"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "tcl"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "tika"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "tinkerpop"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "tomcat"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "tomee"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "traffic server"
test.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
name = "turbine"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "velocity"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "axiom"
test.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
name = "wicket"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "xalan"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "xerces"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "batik"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "fop"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "yetus"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
name = "zookeeper"
test.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
name = "hdfs"
test.AddSkillDictList(name, "apache", "apache " + name , {"apache"})

name = "ansible"
test.AddSkillDictList(name, "red hat", name, {"infrastructure code","red hat"} )
name = "jboss"
test.AddSkillDictList(name, "red hat", name, {"red hat","reliability","performance monitoring"} )

#database
name = "mysql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "postgresql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "mongodb"
test.AddSkillDictList(name, "database", name + " database", {"database","non-relational","document database"})
name = "cassandra"
test.AddSkillDictList(name, "database", name + " database", {"database","non-relational","distributed"})
name = "redis"
test.AddSkillDictList(name, "database", name + " database", {"database","non-relational","in memory database"})
name = "neo4j"
test.AddSkillDictList(name, "database", name + " database", {"database","non-relational","graph database"})
name = "sqlite"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "mariadb"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "elasticsearch"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "couchbase"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "orientdb"
test.AddSkillDictList(name, "database", name + " database", {"database","non-relational"})
name = "firebirdsql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "influxdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "kdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "prometheus"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "chroma"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "pinecone"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "faiss"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "tibero"
test.AddSkillDictList(name, "database", name + " database", {"database"})

#container
name = "docker"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "lxc"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "container linux"
test.AddSkillDictList(name, "container", name ,{"container"} )
name = "portainer"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "messos"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "kubernetes"
test.AddSkillDictList(name, "container", name + " container",{"container"} )

#os
name = "linux"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "solaris"
test.AddSkillDictList(name, "os", name + " technology", {"microsoft","os"} )
name = "vxworks"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )

# cloud
name = "suse"
test.AddSkillDictList(name, "cloud", name + " cloud",{"cloud"} )
name = "snowflake"
test.AddSkillDictList(name, "cloud", name, {"cloud","data warehouse"} )
name = "databricks"
test.AddSkillDictList(name, "cloud", name, {"cloud","data warehouse"} )
name = "citrix"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "salesforce"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "servicenow"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "akamai"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "teradata"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "qualys"
test.AddSkillDictList(name, "security", name, {"cloud","security"} )
name = "heroku"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "alibaba cloud"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "cloudera"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )
name = "firebase"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )

# update here
name = "sensu"
test.AddSkillDictList(name, "cloud", name, {"cloud"} )

name = "talend"
test.AddSkillDictList(name, "reliability", name, {"migration","reliability","etl"} )
name = "sharegate"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
name = "carbonite"
test.AddSkillDictList(name, "reliability", name ,{"migration","backup","reliability"} )
name = "fivetran"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
name = "cloudsfer"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
name = "appdynamics"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability","performance monitoring"} )
name = "informatica"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
name = "turbonomic"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
name = "dynatrace"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability","performance monitoring","network monitoring","network"} )
name = "appdna"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
name = "laplink"
test.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )

# monitoring tool
name = "datadog"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring","network monitoring","network"} )
name = "new relic"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
name = "manageengine"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring","network monitoring","network"} )
name = "solarwinds"
test.AddSkillDictList(name, "security", name,{"reliability","performance monitoring","network monitoring","network","security montioring"} )
name = "elastic apm"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
name = "splunk"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
name = "instana"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
name = "raygun"
test.AddSkillDictList(name, "reliability", name + "performance monitoring",{"reliability","performance monitoring"} )
name = "signoz"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
name = "eg innovations"
test.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )

# pen testing
name = "nmap"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "w3af"
test.AddSkillDictList(name, "security", name +" pen test" , {"security","penetration testing"})
name = "tenable nessus"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "onsecurity"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "wireshark"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing","network","network monitoring","security monitoring"})
name = "zap"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "nikto"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "cobalt"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "rapid7"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
name = "astra"
test.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})

name = "algosec"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})
name = "security onion"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})
name = "elk stack"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})
name = "cisco stealthwatch"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})    
name = "snort"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})    
name = "suricata"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})  
name = "zeek"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"})  
name = "logz.io"
test.AddSkillDictList(name, "security", name , {"security","security monitoring"}) 

name = "nagios"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "prtg"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "zabbix"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "whatsup gold"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "auvik"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "icinga"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "ninjaone"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "cacti"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "spiceworks"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "checkmk"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "kentik"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "netcrunch"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
name = "observium"
test.AddSkillDictList(name, "network", name,{"network","network monitoring"} )

name= "whatgraph"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "grafana"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "kibana"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "tableau"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "infogram"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "sisense"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "looker"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "domo"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} ) 
name= "chartblocks"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "fusioncharts"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "datawrapper"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "zoho analytics"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "qlik"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "qlikview"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )
name= "qliksense"
test.AddSkillDictList(name, "visualization", name ,{"visualization"} )

name = "idrive"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "veeam"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "acronis cyber protect"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "backblaze"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "easeus"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "druva"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "veritas"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "nakivo"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "rubrik"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "r-drive image"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "retrospect solo"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "fbackup"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
name = "shadowprotect"
test.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )

name = "hevo data"
test.AddSkillDictList(name, "etl", name  ,{"etl"})
name = "stitch data"
test.AddSkillDictList(name, "etl", name  ,{"etl"})
name = "fivetran"
test.AddSkillDictList(name, "etl", name  ,{"etl"})

# end


name = "glassfish"
test.AddSkillDictList(name, "network", name + " application servers",{"network"} )
name = "jetty"
test.AddSkillDictList(name, "network", name + " application servers",{"network"} )
name = "wildfly"
test.AddSkillDictList(name, "network", name + " application servers",{"network"} )

# top distributed
name = "mapreduce"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "samza"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "flink"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "heron"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "kudu"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "presto"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name = "centos"
test.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
name= "pyspark"
test.AddSkillDictList(name, "distributed", name , {"distributed","framework"},{"python", "spark"}  )
# mq
name = "rabbitmq"
test.AddSkillDictList(name, "message queue", name , {"message queue"})
name = "ironmq"
test.AddSkillDictList(name, "message queue", name , {"message queue"})
name = "mulesoft"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
name = "pubsub+"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
name = "zeromq"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
name = "tibco"
test.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})

# top ai framework
name = "torch"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
name = "pytorch"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
name = "scikit-learn"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing","visualization","data science"})
name = "tensorflow"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
name = "keras"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
name = "langchain"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "cuda"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "transformers"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "opennn"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "openai"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "pybrain"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "cntk"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "dl4j"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "theano"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "mxnet"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "caffe"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
name = "xgboost"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
name = "opencv"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "viso suite"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "yolo"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "boofcv"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "openvino"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "simplecv"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "deepface"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
name = "spacy"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "xlm"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "chainer"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "gensim"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "nltk"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
name = "matplotlib"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "seaborn"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "pandas"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "numpy"
test.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
name = "streamlit"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "gradio"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "plotly"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "dash"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "panel"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "anvil"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "data mining"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "data modeling"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "data mining"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
name = "language model"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science","natural language processing"})
name = "cadence"
test.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
# top microservices
name = "spring"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "spring boot"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "spring cloud"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "expressjs"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"node.js"})
name = "django"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
name = "django rest"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
name = "go kit"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"go"})
name = "micronaut"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java","groovy","kotlin"})
name = "fastapi"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
name = "helidon"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
name = "lagom"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java","scala"})
name = "quarkus"
test.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})


# top game developer
name = "unreal engine"
test.AddSkillDictList(name, "game", name + " engine", {"game"},{"c++"})
name = "unity"
test.AddSkillDictList(name, "game", name + " engine", {"game"},{"c#"})
name = "godot"
test.AddSkillDictList(name, "game", name + " engine", {"game"},{"c#"})
name = "cocos2d"
test.AddSkillDictList(name, "game", name + " engine", {"game"})
name = "rpg maker"
test.AddSkillDictList(name, "game", name + " engine", {"game"})
name = "vulkan"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game"})
name = "opengl"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game"})
name = "physx"
test.AddSkillDictList(name, "game", name + " physics engine", {"game"})
name = "bullet"
test.AddSkillDictList(name, "game", name + " physics engine", {"game"})
name = "open dynamics engine"
test.AddSkillDictList(name, "game", name + " physics engine", {"game"})
name = "fmod"
test.AddSkillDictList(name, "game", name + " audio framework", {"game"})
name = "glm"
test.AddSkillDictList(name, "game", name + " math framework", {"game"})
name = "assimp"
test.AddSkillDictList(name, "game", name + " library", {"game"})
name = "shader"
test.AddSkillDictList(name, "game", name + " programming", {"game"})
#quantum
name = "tensorflow quantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "cuquantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})


# top test automation frameworks
name = "selenium"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","python","c#"})
name = "appium"
test.AddSkillDictList(name, "testing", name + " framework", {"testing","mobile"})
name = "testng"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","junit"})
name = "junit"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java"})
name = "cucumber"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "testcomplete"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"javascript","python","visual basic"})
name = "robot"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "cypress"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"javascript"})
name = "junit jupiter"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","junit"})
name = "pytest"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"python"})
name = "rspec"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "jenkins"
test.AddSkillDictList(name, "testing", name + " framework", {"testing","devops"})
name = "galen"
test.AddSkillDictList(name, "testing", name + " framework", {"testing","web"})
name = "qtp"
test.AddSkillDictList(name, "testing", name + " framework", {"testing"})
name = "uft"
test.AddSkillDictList(name, "testing", name , {"testing"})

# web crawler
name = "selenium"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"})
name = "selenium base"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"selenium"})
name = "scrapy"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
name = "mechanicalsoup"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
name = "beautiful soup"
test.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})



#top frontend/web
name = "react"
test.AddSkillDictList(name, "backend", name + " framework/library", {"frontend","fullstack","web"})
name = "react.js"
test.AddSkillDictList(name, "backend", name + " framework/library", {"frontend","fullstack","web"})
name = "angular"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = "angular.js"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = "asp.net"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = ".net"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = "asp.net core"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = ".net core"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = "vue.js"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
name = "tailwind css"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "bootstrap"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "mdl"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "bulma"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "semantic ui"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "foundation"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "ulkit"
test.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
name = "node.js"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "nextjs"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "nestjs"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "nuxtjs"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "gatsby"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "flask"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"},{"python"})
name = "django"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"},{"python"})
name = "jquery"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "express"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "wordpress"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "spring boot"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "laravel"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "fastapi"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "svelte"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "ruby on rails"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "blazor"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "symfony"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "deno"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "qwik"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "lit"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "play"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "remix"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "solidjs"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "fastify"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "phoenix"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "drupal"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "codelgniter"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})
name = "rollup.js"
test.AddSkillDictList(name, "web", name + " framework/library", {"web"})

# top backend
name = "spring boot"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"java"})
name = "django"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"python"})
name = "expressjs"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"nodejs"})
name = "laravel"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"php"})
name = "ruby on rails"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"ruby"})
name = "asp.net core"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"c#"})
name = "nestjs"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"typescript","nodejs"})
name = "koajs"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"nodejs"})
name = "phoenix"
test.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"elixir"})


# top mobile development framework
name_list.clear()
name_list.append("flutter")
name_list.append("react native")
name_list.append("angular native")
name_list.append("swiftic")
name_list.append("kotlin")
name_list.append("ionic")
name_list.append("xamarin")
name_list.append("corona")
name_list.append("titanium")
name_list.append("nativescript")
name_list.append("jquery mobile")
for i in range(len(name_list)):
    keyword = name_list[i] + " mobile development framework"
    test.AddSkillDictList(name_list[i], "mobile", keyword, {"mobile"})




# top framework/libraey
name_list.clear()
name_list.append("flutter")
name_list.append("react native")
name_list.append("electron")
name_list.append("qt")
name_list.append("swiftui")
name_list.append("xamarin")
name_list.append("lonic")
name_list.append("gtk")
name_list.append("cordova")
name_list.append("net maui")
name_list.append("tauri")
name_list.append("capacitor")
name_list.append("tidyverse")
name_list.append("quarkus")
name_list.append("ktor")
name_list.append("mfc")
name_list.append("jax")
name_list.append("uno platform")
name_list.append("jdbc")
name_list.append("jpa")
name_list.append("sqlalchemy")
name_list.append("jsp")
name_list.append("extjs")
name_list.append("ejb")
name_list.append("j2ee")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "framework", keyword,{"framework"})

# top blockchain
name = "ethereum"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ibm blockchain"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "hyperledger"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "eos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "corda"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ripple"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "quorum"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "iota"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "neo"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "tezos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "stellar"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "sui network"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "hedera"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "aptos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "flow"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "fantom"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "solana"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "tron"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "binance smart chain"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "icp"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})

name = "fleet"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "intellij"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "pycharm"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "webstorm"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "phpstorm"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "rider"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "clion"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "datalore"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "datagrip"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "rubymine"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "goland"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "resharper c++"
test.AddSkillDictList(name, "ide", name ,{"ide"} )
name = "rustrover"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "aqua"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "anaconda"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "jupyter"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "eclipse"
test.AddSkillDictList(name, "ide", name ,{"ide"})
name = "vert.x"
test.AddSkillDictList(name, "ide", name ,{"ide"})

name = "gitlab"
test.AddSkillDictList(name, "git", name ,{"continuous integration and continuous deployment","git","devops"} )
name = "bitbucket"
test.AddSkillDictList(name, "git", name ,{"continuous integration and continuous deployment","git","devops"} )
name = "github"
test.AddSkillDictList(name, "git", name ,{"git","devops"} )
name = "preforce"
test.AddSkillDictList(name, "git", name ,{"git","devops"} )

# cl/cd

test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "buddy"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "teamcity"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "circleci"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "tracisci"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "semaphore"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "puppet"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
name = "gradle"
test.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )



name= "xml"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "json"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "binary"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "xpath"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "query"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "ajax"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "csv"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "hexadecimal"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "xsl"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "wsdl"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "xslt"
test.AddSkillDictList(name, "format", name + " format",{"format"} )
name= "xsd"
test.AddSkillDictList(name, "format", name + " format",{"format"} )


# infa code
name = "terraform"
test.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )
name = "spacelift"
test.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )


name = "esri"
test.AddSkillDictList(name, "software or tool", name + " technology",{"arcgis"}  )


name = "control m"
test.AddSkillDictList(name, "software or tool", name)
name = "loadrunner"
test.AddSkillDictList(name, "software or tool", name)
name = "tibco ems"
test.AddSkillDictList(name, "software or tool", name)
name = "akka"
test.AddSkillDictList(name, "software or tool", name)
name = "logstash"
test.AddSkillDictList(name, "software or tool", name)
name = "ubuntu"
test.AddSkillDictList(name, "software or tool", name)
name = "nginx"
test.AddSkillDictList(name, "load balancer", name,{"load balancer","architecture"})
name="ccna"
test.AddSkillDictList(name, "cisco", name ,{"security","cisco"} )
name="nessus"
test.AddSkillDictList(name, "security", name ,{"security"} )
name="ssl"
test.AddSkillDictList(name, "security", name ,{"security","network","web"} )
name= "gis"
test.AddSkillDictList(name, "other", name )
name = "dlp"
test.AddSkillDictList(name, "security", name ,{"security"} )
name = "dbms"
test.AddSkillDictList(name, "database", name ,{"database"})
                      
name = "ip address"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "mac address"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "firewall"
test.AddSkillDictList(name, "network", name ,{"network","security"} )
name = "domain name system"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "area network"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "storage area network"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "dhcp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "network protocol"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "network topology"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "network devices"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "broadcasting"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "subnet"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "subnet mask"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "soap"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "ospf"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "access control"
test.AddSkillDictList(name, "security", name ,{"network","architecture","security"} )
name = "xmpp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "eigrp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "nfs"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "vrrp"
test.AddSkillDictList(name, "network", name ,{"network"} )
name = "mpls"
test.AddSkillDictList(name, "network", name ,{"network"} )
                      


name = "array"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "list"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "tuple"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "stack"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "queue"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "tree"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "graph"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "trie"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
name = "hash table"
test.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )

name = "abstraction"
test.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
name = "encapsulation"
test.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
name = "inheritance"
test.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
name = "polymorphism"
test.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
name = "object"
test.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
name = "class"
test.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )

name = "factory"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "builder"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "dependency injection"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "lazy"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "object pool"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "multition"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "raii"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "prototype"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "singleton"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "visitor"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "join"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "lock"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "mvc"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "actors"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "component"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "strategy"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "observer"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "adapter"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "state"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "creational"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "structural"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "behavioral"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "decorator"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "facade"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
name = "mvvm"
test.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )


name = "monolithic system"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"} )
name = "distributed system"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "structured analysis"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "structured design"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "system design strategy"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "database sharding"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "scaling"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "caching"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "ooad"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "public cloud"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "private cloud"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "hybrid cloud"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "file storage"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "block storage"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "object storage"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})
name = "availability"
test.AddSkillDictList(name, "system design", name + " system design",{"system design","architecture"})

name= "software development life cycle"
test.AddSkillDictList(name, "other", name ,{"architecture"})
name= "agile"
test.AddSkillDictList(name, "other", name ,{"architecture"})

name = "postman"
test.AddSkillDictList(name, "software or tool", name,{"testing","web"} )

skill_set ={"c++","java","python","c","c#","javascript","typescript","php","swift","kotlin","dart","go","ruby","scala","rust","racket","erlang","elixir"}
#test.AddGroupDictList("leetcode",skill_set)




test.ExportSkillDictList()
test.ExportGroupDictList()

In [9]:
test.exact_match_replace_dict_list["aws"]="amazon web services"
test.exact_match_replace_dict_list["tdd"]="testing"
test.exact_match_replace_dict_list["webdriver"]="web crawler"
test.exact_match_replace_dict_list["vbnet"]="visual basic .net"
test.exact_match_replace_dict_list["vb.net"]="visual basic .net"
test.exact_match_replace_dict_list["vb"]="visual basic"
test.exact_match_replace_dict_list["containerization"]="container"
test.exact_match_replace_dict_list["html5"]="html"
test.exact_match_replace_dict_list["svn"]="subversion"
test.exact_match_replace_dict_list["version control"]="git"
test.exact_match_replace_dict_list["nosql"]="non-relational"
test.exact_match_replace_dict_list["rdbms"]="relational"
test.exact_match_replace_dict_list["unity3d"]="unity"
test.exact_match_replace_dict_list["mssql"]="microsoft sql"
test.exact_match_replace_dict_list["shaders"]="shader"
test.exact_match_replace_dict_list["redhat"]="red hat"
test.exact_match_replace_dict_list["powerbi"]="power bi"
test.exact_match_replace_dict_list["uat"]="testing"
test.exact_match_replace_dict_list["objective-c"]="objective c"
test.exact_match_replace_dict_list["objective-c"]="objective c"


test.partial_match_replace_dict_list["ms"]="microsoft"
test.partial_match_replace_dict_list["window"]="windows"
test.partial_match_replace_dict_list["databases"]="database"
test.partial_match_replace_dict_list["website"]="web"
test.partial_match_replace_dict_list["solarwinds"]="solarwind"

test.ExportMatchReplaceDictList()



In [7]:
f = open("sample skill.txt", "r")
skills = set()
for c in f:
    c = c.replace("\n", "")
    skills.add(c)
f.close


for s in skills:
    s = s.replace("/", "-")
    s = test.Filter(s)
    if s != "":
        found = test.Search(s)
        if found == False:
            #if len(words) == 1:
            test.not_found_set.add(s)
                
                
test.ExportNotFoundSet()

In [10]:
f = open("sample skill.txt", "r")
skills = set()
for c in f:
    c = c.replace("\n", "")
    skills.add(c)
f.close
result = test.GenerateLearningResource(None, skills, None)

{'backend', 'kafka', 'sqlite', 'rust', 'exchange', 'vcenter', 'ruby on rails', 'veeam', 'devops', 'mvc', 'strategy', 'gitlab', 'db2', 'eigrp', 'oracle cloud', 'felix', 'system design', 'clojure', 'informatica', 'typescript', 'query', 'heroku', 'powershell', 'airflow', 'react.js', 'grafana', 'epm', 'ldap', 'jenkins', 'camel', 'continuous integration and continuous deployment', 'orc', 'c', 'emr', 'mulesoft', 'c++', 'container', 'amazon web services', 'data mining', 'git', 'elixir', 'puppet', 'data science', 'sap basis', 'apple', 'word', 'fastapi', 'qlikview', 'logging', 'ospf', 'splunk', 'j2ee', 'kibana', 'node.js', 'windows server', 'red hat', 'subversion', 'scala', 'aspnet mvc', 'jupyter', 'mariadb', 'esri', 'stack', 'object', 'jdbc', 'cucumber', 'hybrid cloud', 'windows', 'swift', 'jboss', 'foundation', 'jquery', 'reliability', 'migration', 'os', 'tibero', '.net core', 'blockchain', 'dynatrace', 'hadoop', 'database', 'alibaba cloud', 'talend', 'microsoft', 'public cloud', 'ajax', 'cen

In [ ]:
   for word in words:
                        s_found = False
                        g_found = False
                        for sdl in test.skill_dict_list:
                            if nlp.vocab[word].has_vector == True and nlp.vocab[sdl].has_vector ==True:
                                doc1 = nlp(sdl)
                                doc2 = nlp(word)
                                similarity_score = doc1.similarity(doc2)
                                if similarity_score >= 0.9:
                                    s_found = True
                                    print(sdl,":",s)
                                    prepare.add(sdl)
                        for gdl in test.group_dict_list:
                            if nlp.vocab[word].has_vector == True and nlp.vocab[gdl].has_vector ==True:
                                doc1 = nlp(gdl)
                                doc2 = nlp(word)
                                similarity_score = doc1.similarity(doc2)
                                if similarity_score >= 0.9:
                                    g_found = True
                                    print(gdl,":",s)
                                    prepare.add(gdl)
                        if s_found == False and g_found == False:
                            test.not_found_set.add(s)

In [8]:

data_set = ""
f = open("not found.txt", "r")
for c in f:
    c = c.replace("\n", "")
    c= c.lower()
    c = c.replace("/", "-")
    if c.find('(') != -1:
        c = c.split("(")[0]
        c = c.rsplit()[0]
    data_set += c
    data_set += " "
f.close()
split_it = data_set.split() 
counter = {}
for w in split_it:
    if w not in counter:
        counter[w] = 0
    counter[w] +=1
my_keys = sorted(counter, key=counter.get, reverse=True)
print(my_keys)

with open('ignore2.txt', 'w') as f:
    for k in sorted(counter, key=counter.get, reverse=True) :
        f.write(k)
        f.write("\n")
    f.close()





['data', 'design', 'analysis', 'test', 'systems', 'software', 'development', 'technical', 'system', 'it', 'engineering', 'technology', 'application', 'planning', 'infrastructure', 'automation', 'services', 'project', 'user', 'integration', 'documentation', 'research', 'product', 'and', 'quality', 'computer', 'deployment', 'applications', 'unix', 'analytics', 'visual', 'maintenance', 'skills', 'tracking', 'troubleshooting', 'information', 'networking', 'industry', 'production', 'modeling', 'video', 'servers', 'office', 'operating', 'operations', 'trading', 'process', 'experience', 'solutions', 'threat', 'intelligence', 'hardware', 'monitoring', 'desktop', 'tuning', 'knowledge', 'recovery', 'scripting', 'learning', 'coding', 'public', 'processing', 'workflow', 'certified', 'risk', 'predictive', 'payment', 'service', 'detection', 'waterfall', 'computing', 'encryption', 'improvement', 'architectural', 'automated', 'assessment', 'cases', 'cyber', 'statistical', 'configuration', 'disaster', 

Document converted successfully.
